*10/23/2023 Melody*

Detect fitness devices using [YOLOv8](https://docs.ultralytics.com/) model.
1. Prepare data for training and testing
    - (done) Automate the rotation of 3D fitness device models in Blender;
    - (half-way) Automate the screenshot process of each angle;
    - (done) Semi-automate the process of data labelling.
2. Train the YOLOv8 model with customized dataset.
3. Detect fitness devices from videos.

### Overall workflow

**1. Object Detection (YOLOv8)**

2. Human figure segmentation (Segment Anything)

3. Pose Estimation (MoveNet)

4. Pose Estimation Correction

5. Define and classify fitness poses

6. Output the text description

## 1. Prepare data for training and testing

- Automate the rotation of 3D fitness device models in Blender;

- Automate the screenshot process of each angle;

(Open Blender --> Scripting --> run the code there, not here)

Problems remain: it's a limitation since all the scripts passed via command line argument 
are executed at command line processing time, prior to any OpenGL draw is done.

In [ ]:
import bpy
import math
import numpy as np

# https://www.youtube.com/watch?v=XqX5wh4YeRw
so = bpy.data.objects.get("defaultMaterial.001")

filepath1 = "//output/x_axis/"

# Why Blender could not save screenshots after drawing
# https://blender.stackexchange.com/questions/82936/blender-creates-screenshot-using-python-script-but-not-when-running-in-backgroun
'''
it's a limitation since all the scripts passed via command line argument 
are executed at command line processing time, prior to any OpenGL draw is done.
'''

# Rotate object by x axis
for count in range(5, 181, 5):
    rad = count * math.pi / 180
    so.rotation_euler[0] += rad
    bpy.context.view_layer.update()
    # Take snapshot
    filepath1 = "//output/x_axis/"
    filepath1 += str(count)
    filepath1 += ".png"
    # https://docs.blender.org/api/current/bpy.ops.screen.html#bpy.ops.screen.screenshot
    bpy.ops.screen.screenshot(filepath=filepath1)

    
filepath2 = "//output/y_axis/"

# Rotate object by y axis
for count in range(5, 181, 5):
    rad = count * math.pi / 180
    so.rotation_euler[1] += rad
    # Take snapshot
    filepath2 = "//output/y_axis/"
    filepath2 += str(count)
    filepath2 += ".png"
    # https://shuvit.org/python_api/bpy.ops.screen.html
    bpy.ops.screen.screenshot(filepath=filepath2)
    
    
    
filepath3 = "//output/z_axis/"

# Rotate object by z axis
for count in range(5, 181, 5):
    rad = count * math.pi / 180
    so.rotation_euler[2] += rad
    # Take snapshot
    filepath3 = "//output/z_axis/"
    filepath3 += str(count)
    filepath3 += ".png"
    bpy.ops.screen.screenshot(filepath=filepath3)

## 2. Train the YOLOv8 model with customized dataset

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data='/Users/melodyjoyou/Gymnet/Object_Detection/YOLOv8/Dataset/Blender_data/data.yaml', epochs=50, imgsz=640)